In [8]:
import json
import numpy as np
import obi_one as obi
import os
import pandas as pd
from conntility.connectivity import ConnectivityMatrix

In [9]:
input_root = "/Users/pokorny/Data/Circuits"
cmat_root = "/Users/pokorny/Data/ConnectivityMatrices"
output_root = "../../../obi-output/extracted_small_microcircuits_simplex"

In [10]:
# Load nbS1-O1 circuit with connectivity matrix
circuit = obi.Circuit(name="nbS1-O1",
                      path=os.path.join(input_root, "nbS1-O1", "circuit_config_postfix2.json"),
                      matrix_path=os.path.join(cmat_root, "nbS1-O1", "connectivity_matrix.h5"))
display(circuit.connectivity_matrix.matrix)

<COOrdinate sparse matrix of dtype 'int64'
	with 87457567 stored elements and shape (211712, 211712)>

In [23]:
# nbS1-O1 - Central neurons in connectivity
df_sel = pd.read_pickle("Top10CentralNeurons_Layer_Hex.pkl")  # (Contains neuron IDs relative to nbS1-HEXn)
df_opt = pd.read_pickle("Top10CentralNeurons_Layer_Hex_OptIDX.pkl")

# Load HEXi connectivity matrix
base_name = "nbS1"
hex_sel = "hex0"
layers = range(1, 7)
cmat = ConnectivityMatrix.from_h5(os.path.join(cmat_root, f"{base_name}-{hex_sel.upper()}", "connectivity_matrix.h5"))
ntab = cmat.vertices.set_index("node_ids")

# Load ID mapping w.r.t. parent circuit nbS1-O1
id_map_file = os.path.join(input_root, f"{base_name}-{hex_sel.upper()}", "id_mapping.json")
with open(id_map_file, "r") as f:
    id_map = json.load(f)
assert np.array_equal(id_map["S1nonbarrel_neurons"]["new_id"], ntab.index.values)

nid_selection = []  # Selected neuron IDs for nbS1-O1 circuit
for _lay in layers:
    opt_idx = df_opt.loc[(hex_sel, _lay)]["pos"]
    node_id = df_sel[("hex0", _lay)].iloc[opt_idx].tolist()
    nid_selection.append(id_map["S1nonbarrel_neurons"]["old_id"][node_id])

# Check selection
cmat = circuit.connectivity_matrix
sel_tab = cmat.vertices.loc[nid_selection]
assert np.array_equal(sel_tab["layer"].astype(int), layers)
display(sel_tab)

,node_ids,etype,layer,mtype,synapse_class,x,y,z
33691,33691,cNAC,1,L1_NGC-DA,INH,4683.919729,-1387.374371,-1885.088008
151361,151361,cNAC,2,L23_CHC,INH,4417.154568,-1455.019186,-1899.847804
2625,2625,cADpyr,3,L3_TPC:A,EXC,4457.265464,-1316.485898,-2006.089937
116904,116904,cADpyr,4,L4_UPC,EXC,4124.450870,-1479.844101,-2191.272675
171462,171462,cADpyr,5,L5_TPC:C,EXC,3848.582449,-1385.443785,-2703.829151
79088,79088,cADpyr,6,L6_IPC,EXC,3592.676277,-1025.078101,-2898.148159


In [24]:
# Set up simplex neuron sets
N = 10  # Max. number of neurons
dim = 10
simplex_pos = "source"
simplex_neuron_sets = obi.SimplexNeuronSet(
    central_neuron_id=nid_selection,
    dim=dim,
    central_neuron_simplex_position=simplex_pos,
    subsample=True,
    seed=0,
    n_count_max=N,
    simplex_type="directed",
    property_filter=obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(),
)

virtual_sources_to_ignore = ("external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical", )
output_path = os.path.join(output_root, f"dim{dim}-{simplex_pos}")

In [25]:
circuit_extractions_form = obi.CircuitExtractions(
                    initialize=obi.CircuitExtractions.Initialize(
                        circuit=circuit,
                        run_validation=False,
                        do_virtual=True,
                        create_external=True,
                        virtual_sources_to_ignore=virtual_sources_to_ignore,
                    ),
                    neuron_set=simplex_neuron_sets,
)

coupled_scan = obi.CoupledScan(form=circuit_extractions_form, output_root=output_path, coordinate_directory_option="ZERO_INDEX")

In [26]:
# Display the intermediary data
coupled_scan.multiple_value_parameters(display=True)
coupled_scan.coordinate_parameters(display=True)
coupled_scan.coordinate_instances(display=True)

[2025-06-19 10:55:09,963] INFO: 
MULTIPLE VALUE PARAMETERS
[2025-06-19 10:55:09,966] INFO: neuron_set.central_neuron_id: [33691, 151361, 2625, 116904, 171462, 79088]
[2025-06-19 10:55:09,967] INFO: 
COORDINATE PARAMETERS
[2025-06-19 10:55:09,968] INFO: neuron_set.central_neuron_id: 33691
[2025-06-19 10:55:09,969] INFO: neuron_set.central_neuron_id: 151361
[2025-06-19 10:55:09,970] INFO: neuron_set.central_neuron_id: 2625
[2025-06-19 10:55:09,970] INFO: neuron_set.central_neuron_id: 116904
[2025-06-19 10:55:09,971] INFO: neuron_set.central_neuron_id: 171462
[2025-06-19 10:55:09,971] INFO: neuron_set.central_neuron_id: 79088
[2025-06-19 10:55:09,973] INFO: 
COORDINATE INSTANCES
[2025-06-19 10:55:09,974] INFO: CircuitExtraction(idx=0, scan_output_root=PosixPath('.'), coordinate_output_root=PosixPath('.'), single_coordinate_scan_params=SingleCoordinateScanParams(type='SingleCoordinateScanParams', scan_params=[SingleValueScanParam(type='SingleValueScanParam', location_list=['neuron_set', 'c

[CircuitExtraction(idx=0, scan_output_root=PosixPath('.'), coordinate_output_root=PosixPath('.'), single_coordinate_scan_params=SingleCoordinateScanParams(type='SingleCoordinateScanParams', scan_params=[SingleValueScanParam(type='SingleValueScanParam', location_list=['neuron_set', 'central_neuron_id'], value=33691)], nested_coordinate_subpath_str=PosixPath('.')), type='CircuitExtraction', initialize=Initialize(type='CircuitExtractions.Initialize', circuit=Circuit(type='Circuit', name='nbS1-O1', path='/Users/pokorny/Data/Circuits/nbS1-O1/circuit_config_postfix2.json', matrix_path='/Users/pokorny/Data/ConnectivityMatrices/nbS1-O1/connectivity_matrix.h5'), run_validation=False, do_virtual=True, create_external=True, virtual_sources_to_ignore=('external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',)), neuron_set=SimplexNeuronSet(type='SimplexNeuronSet', random_sample=None, random_seed=0, node_population=None, property_filter=NoFilter, node_sets=(), central_neuron_id=33691, dim=10, c

In [27]:
coupled_scan.execute(processing_method='run')

[2025-06-19 10:55:11,608] INFO: None
[2025-06-19 10:56:03,215] INFO: COMPUTE list of simplices by dimension
> Dimension not attained using dimension 5 instead.
Extracting subcircuit from 'nbS1-O1'
Copying morphologies for population 'S1nonbarrel_neurons' (6)


Copying containerized .h5 morphologies: 100%|████████████████████████████████████████████████████| 6/6 [00:00<00:00, 676.06it/s]

Copied 6 morphologies into container (0 already existed)
Copying 5 biophysical neuron models (.hoc) for population 'S1nonbarrel_neurons' (6)
Copying mod files
Extraction DONE


[2025-06-19 11:00:03,294] INFO: COMPUTE list of simplices by dimension
> Dimension not attained using dimension 6 instead.
Extracting subcircuit from 'nbS1-O1'
Copying morphologies for population 'S1nonbarrel_neurons' (7)


Copying containerized .h5 morphologies: 100%|████████████████████████████████████████████████████| 7/7 [00:00<00:00, 799.98it/s]

...

Copied 10 morphologies into container (0 already existed)
Copying 2 biophysical neuron models (.hoc) for population 'S1nonbarrel_neurons' (10)
Copying mod files
Extraction DONE


[2025-06-19 11:15:42,559] INFO: COMPUTE list of simplices by dimension
> Dimension not attained using dimension 6 instead.
Extracting subcircuit from 'nbS1-O1'
Copying morphologies for population 'S1nonbarrel_neurons' (7)


Copying containerized .h5 morphologies: 100%|████████████████████████████████████████████████████| 7/7 [00:00<00:00, 809.15it/s]

Copied 7 morphologies into container (0 already existed)
Copying 1 biophysical neuron models (.hoc) for population 'S1nonbarrel_neurons' (7)
Copying mod files
Extraction DONE
[2025-06-19 11:18:47,017] INFO: create_bbp_workflow_campaign_config() not yet complete.


{0: None, 1: None, 2: None, 3: None, 4: None, 5: None}

In [29]:
# Check circuits
from bluepysnap import Circuit
from connalysis.network import topology

for _idx, _inst in enumerate(coupled_scan.coordinate_instances()):
    cfg = coupled_scan.output_root / str(_idx) / "circuit_config.json"
    c = Circuit(cfg)
    nodes = c.nodes['S1nonbarrel_neurons']
    mtypes = np.unique(nodes.get(properties=['mtype']), return_counts=True)
    mt_hist = [f"{_c}x{_mt}" for _mt, _c in zip(*mtypes)]
    print(f"L{layers[_idx]}: {nodes.size} neurons, {c.edges["S1nonbarrel_neurons__S1nonbarrel_neurons__chemical"].size} synapses, m-types: {mt_hist}")


L1: 6 neurons, 107 synapses, m-types: ['1xL1_NGC-DA', '1xL3_TPC:C', '1xL4_TPC', '1xL5_TPC:A', '1xL5_TPC:B', '1xL6_TPC:A']
L2: 7 neurons, 150 synapses, m-types: ['1xL23_CHC', '4xL3_TPC:A', '2xL3_TPC:C']
L3: 6 neurons, 99 synapses, m-types: ['1xL2_TPC:B', '1xL3_TPC:A', '3xL4_TPC', '1xL5_TPC:B']
L4: 6 neurons, 111 synapses, m-types: ['1xL4_UPC', '3xL5_TPC:A', '1xL5_TPC:B', '1xL6_UPC']
L5: 10 neurons, 236 synapses, m-types: ['2xL5_TPC:A', '4xL5_TPC:B', '1xL5_TPC:C', '1xL6_IPC', '2xL6_TPC:A']
L6: 7 neurons, 163 synapses, m-types: ['2xL6_HPC', '1xL6_IPC', '3xL6_TPC:A', '1xL6_UPC']
